In [67]:
import numpy as np
import pandas as pd
import pandas_profiling


import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

# utilities
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots


def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value':val,
            'All':dataset[dataset[feature] == val].count()[feature],
            'Good':dataset[(dataset[feature] == val) & (dataset[target].astype(int) == 0)].count()[feature],
            'Bad':dataset[(dataset[feature] == val) & (dataset[target].astype(int) == 1)].count()[feature],
        })
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['Prcnr_Good'] = dset['Good'] / dset['All']
    dset['Prcnr_Bad'] = dset['Bad'] / dset['All']
    dset['WOE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WOE':{np.inf: 0, -np.inf:0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WOE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values( by = 'WOE')
    
    return dset,iv


In [2]:
df = pd.read_csv('Family Income and Expenditure/Family Income and Expenditure.csv')

In [3]:
df.head()

,Total Household Income,Region,Total Food Expenditure,Main Source of Income,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,480332,CAR,117848,Wage/Salaries,0,42140,38300,24676,16806,3325,...,1,1,0,0,0,2,1,0,0,1
1,198235,CAR,67766,Wage/Salaries,0,17329,13008,17434,11073,2035,...,0,1,0,0,0,3,1,0,0,2
2,82785,CAR,61609,Wage/Salaries,1,34182,32001,7783,2590,1730,...,0,0,0,0,0,0,0,0,0,0
3,107589,CAR,78189,Wage/Salaries,0,34030,28659,10914,10812,690,...,0,0,0,0,0,1,0,0,0,0
4,189322,CAR,94625,Wage/Salaries,0,34820,30167,18391,11309,1395,...,1,0,0,0,0,3,0,0,0,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41544 entries, 0 to 41543
Data columns (total 60 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   Total Household Income                         41544 non-null  int64 
 1   Region                                         41544 non-null  object
 2   Total Food Expenditure                         41544 non-null  int64 
 3   Main Source of Income                          41544 non-null  object
 4   Agricultural Household indicator               41544 non-null  int64 
 5   Bread and Cereals Expenditure                  41544 non-null  int64 
 6   Total Rice Expenditure                         41544 non-null  int64 
 7   Meat Expenditure                               41544 non-null  int64 
 8   Total Fish and  marine products Expenditure    41544 non-null  int64 
 9   Fruit Expenditure                              41544 non-null

In [5]:
df.select_dtypes(include=['int64'])

,Total Household Income,Total Food Expenditure,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,Vegetables Expenditure,Restaurant and hotels Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,480332,117848,0,42140,38300,24676,16806,3325,13460,3000,...,1,1,0,0,0,2,1,0,0,1
1,198235,67766,0,17329,13008,17434,11073,2035,7833,2360,...,0,1,0,0,0,3,1,0,0,2
2,82785,61609,1,34182,32001,7783,2590,1730,3795,4545,...,0,0,0,0,0,0,0,0,0,0
3,107589,78189,0,34030,28659,10914,10812,690,7887,6280,...,0,0,0,0,0,1,0,0,0,0
4,189322,94625,0,34820,30167,18391,11309,1395,11260,6400,...,1,0,0,0,0,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41539,119773,44875,1,23675,21542,1476,6120,1632,3882,1805,...,0,0,0,0,0,1,0,0,0,0
41540,137320,31157,1,2691,1273,1886,4386,1840,3110,9090,...,0,0,0,0,0,3,0,0,0,0
41541,133171,45882,2,28646,27339,480,4796,1232,3025,3330,...,0,0,0,0,0,1,0,0,0,0
41542,129500,81416,1,29996,26655,2359,17730,2923,7951,13660,...,0,0,0,0,0,2,0,0,0,0


In [6]:
df.select_dtypes(include=['object'])

,Region,Main Source of Income,Household Head Sex,Household Head Marital Status,Household Head Highest Grade Completed,Household Head Job or Business Indicator,Household Head Occupation,Household Head Class of Worker,Type of Household,Type of Building/House,Type of Roof,Type of Walls,Tenure Status,Toilet Facilities,Main Source of Water Supply
0,CAR,Wage/Salaries,Female,Single,Teacher Training and Education Sciences Programs,With Job/Business,General elementary education teaching professi...,Worked for government/government corporation,Extended Family,Single house,"Strong material(galvanized,iron,al,tile,concre...",Strong,Own or owner-like possession of house and lot,"Water-sealed, sewer septic tank, used exclusiv...","Own use, faucet, community water system"
1,CAR,Wage/Salaries,Male,Married,Transport Services Programs,With Job/Business,Transport conductors,Worked for private establishment,Single Family,Single house,"Strong material(galvanized,iron,al,tile,concre...",Strong,Own or owner-like possession of house and lot,"Water-sealed, sewer septic tank, used exclusiv...","Own use, faucet, community water system"
2,CAR,Wage/Salaries,Male,Married,Grade 3,With Job/Business,Farmhands and laborers,Worked for private establishment,Single Family,Single house,"Light material (cogon,nipa,anahaw)",Light,Own or owner-like possession of house and lot,"Water-sealed, sewer septic tank, shared with o...","Shared, faucet, community water system"
3,CAR,Wage/Salaries,Male,Married,Elementary Graduate,With Job/Business,Rice farmers,Employer in own family-operated farm or business,Single Family,Single house,"Light material (cogon,nipa,anahaw)",Light,Own or owner-like possession of house and lot,Closed pit,"Own use, faucet, community water system"
4,CAR,Wage/Salaries,Male,Married,Elementary Graduate,With Job/Business,General managers/managing proprietors in trans...,Self-employed wihout any employee,Single Family,Single house,"Light material (cogon,nipa,anahaw)",Quite Strong,Own or owner-like possession of house and lot,"Water-sealed, sewer septic tank, used exclusiv...","Own use, faucet, community water system"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41539,XII - SOCCSKSARGEN,Enterpreneurial Activities,Male,Married,Elementary Graduate,With Job/Business,Rice farmers,Self-employed wihout any employee,Extended Family,Single house,Mixed but predominantly strong materials,Strong,Own or owner-like possession of house and lot,Open pit,"Own use, tubed/piped deep well"
41540,XII - SOCCSKSARGEN,Enterpreneurial Activities,Male,Married,Elementary Graduate,With Job/Business,Rice farmers,Self-employed wihout any employee,Single Family,Single house,"Strong material(galvanized,iron,al,tile,concre...",Strong,Own or owner-like possession of house and lot,Open pit,"Own use, tubed/piped deep well"
41541,XII - SOCCSKSARGEN,Enterpreneurial Activities,Male,Married,Grade 1,With Job/Business,Rice farmers,Self-employed wihout any employee,Single Family,Single house,"Light material (cogon,nipa,anahaw)",Strong,"Own house, rent-free lot with consent of owner",Open pit,"Own use, tubed/piped deep well"
41542,XII - SOCCSKSARGEN,Enterpreneurial Activities,Male,Widowed,First Year High School,With Job/Business,Rice farmers,Self-employed wihout any employee,Extended Family,Single house,"Strong material(galvanized,iron,al,tile,concre...",Strong,"Own house, rent-free lot without consent of owner",Open pit,Dug well


In [7]:
null_data = df[df.isnull().any(axis=1)]
print(null_data.shape)

(7536, 60)


In [8]:
#df = df.fillna(df.mean())

In [73]:
dataset = df.copy()

In [74]:
dataset = dataset.drop(['Household Head Occupation', 'Household Head Class of Worker', 'Type of Roof', 'Type of Walls', 'Toilet Facilities', 'Main Source of Water Supply'], axis= 1)

In [75]:
#dataset = pd.get_dummies(dataset, columns=['Region','Main Source of Income','Household Head Sex', 'Household Head Marital Status', 'Household Head Highest Grade Completed', 'Household Head Job or Business Indicator', 'Type of Household', 'Type of Building/House', 'Tenure Status'])
dataset = dataset.drop(columns=['Region','Main Source of Income','Household Head Sex', 'Household Head Marital Status', 'Household Head Highest Grade Completed', 'Household Head Job or Business Indicator', 'Type of Household', 'Type of Building/House', 'Tenure Status'])

In [76]:
dataset.head()

,Total Household Income,Total Food Expenditure,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,Vegetables Expenditure,Restaurant and hotels Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,480332,117848,0,42140,38300,24676,16806,3325,13460,3000,...,1,1,0,0,0,2,1,0,0,1
1,198235,67766,0,17329,13008,17434,11073,2035,7833,2360,...,0,1,0,0,0,3,1,0,0,2
2,82785,61609,1,34182,32001,7783,2590,1730,3795,4545,...,0,0,0,0,0,0,0,0,0,0
3,107589,78189,0,34030,28659,10914,10812,690,7887,6280,...,0,0,0,0,0,1,0,0,0,0
4,189322,94625,0,34820,30167,18391,11309,1395,11260,6400,...,1,0,0,0,0,3,0,0,0,1


In [196]:
y = dataset['Total Household Income']
X = dataset.drop(['Total Household Income'], axis = 1)
features_start = X.columns

In [197]:
lst_bin = []
lst_ct = []
for col in X:
    if len(set(list(X[col]))) == 2:
        lst_bin.append(col)
    else:
        lst_ct.append(col)

In [198]:
y.describe()

count    4.154400e+04
mean     2.475556e+05
std      2.868805e+05
min      1.128500e+04
25%      1.048950e+05
50%      1.640795e+05
75%      2.911385e+05
max      1.181599e+07
Name: Total Household Income, dtype: float64

In [199]:
lst_cl = []
lst_ln = []
for col in X[lst_ct].columns:
    if (1 in set(list(X[lst_ct][col]))) | (0 in set(list(X[lst_ct][col]))):
        lst_cl.append(col)
    else:
        lst_ln.append(col)

In [200]:
scaler = StandardScaler()
# 1 способ
#X[lst_ct] = scaler.fit_transform(X[lst_ct])
# 2 способ
X[lst_ln] = np.log(X[lst_ln])
X[lst_cl] = scaler.fit_transform(X[lst_cl])
# 3 способ
#for col in X.columns:
#    X[col] = (X[col] - X[col].min())/ (X[col].max() - X[col].min())
X.head()

,Total Food Expenditure,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,Vegetables Expenditure,Restaurant and hotels Expenditure,Alcoholic Beverages Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,11.677151,-0.635265,1.262875,1.714337,1.346586,0.803456,0.260808,2.561435,-0.524641,-0.499018,...,1.128109,1.421585,-0.289333,-0.234197,-0.219025,0.060497,0.926044,-0.381377,-0.103159,1.271693
1,11.123816,-0.635265,-0.579574,-0.442380,0.656715,0.069595,-0.173577,0.856353,-0.551640,-0.057518,...,-0.734024,1.421585,-0.289333,-0.234197,-0.219025,0.702300,0.926044,-0.381377,-0.103159,3.062520
2,11.028563,0.842586,0.671919,1.177204,-0.262638,-1.016284,-0.276281,-0.367233,-0.459465,-0.374846,...,-0.734024,-0.668383,-0.289333,-0.234197,-0.219025,-1.223108,-0.425875,-0.381377,-0.103159,-0.519135
3,11.266884,-0.635265,0.660632,0.892223,0.035620,0.036185,-0.626482,0.872716,-0.386273,-0.278268,...,-0.734024,-0.668383,-0.289333,-0.234197,-0.219025,-0.581305,-0.425875,-0.381377,-0.103159,-0.519135
4,11.457677,-0.635265,0.719296,1.020814,0.747878,0.099804,-0.389086,1.894795,-0.381211,-0.020727,...,1.128109,-0.668383,-0.289333,-0.234197,-0.219025,0.702300,-0.425875,-0.381377,-0.103159,1.271693


In [201]:
def info(df):
    print(f"5-квантиль {df.sort_values(ascending = True).quantile(q = 0.05)}")
    print(f"25-квантиль {df.sort_values(ascending = True).quantile(q = 0.25)}")
    print(f"75-квантиль {df.sort_values(ascending = True).quantile(q = 0.75)}")
    print(f"95-квантиль {df.sort_values(ascending = True).quantile(q = 0.95)}")
    print(f"Медиана {df.sort_values(ascending = True).median()}")
    print(f"Среднее {df.sort_values(ascending = True).mean()}")
    print(f"MAX {df.sort_values(ascending = True).max()}")
    print(f"MIN {df.sort_values(ascending = True).min()}")

info(y)

5-квантиль 56072.15
25-квантиль 104895.0
75-квантиль 291138.5
95-квантиль 692298.0499999996
Медиана 164079.5
Среднее 247555.58480165608
MAX 11815988
MIN 11285


In [202]:
xx = y.sort_values(ascending = True).quantile(q = 0.95)
xy = y.sort_values(ascending = True).median()
y = [x if x < xx else xy for x in y]

In [203]:
# 1 способ
#y = scaler.fit_transform(pd.DataFrame(y, columns = ['y']))
# 2 способ
y = np.log(y)
# 3 способ
#y = (pd.DataFrame(y) - pd.DataFrame(y).min())/ (pd.DataFrame(y).max() - pd.DataFrame(y).min())

In [204]:
y

array([13.08223281, 12.19720847, 11.32400216, ..., 11.7993893 ,
       11.77143616, 11.76444654])

# Отбор переменных с standalone importance > 0

In [205]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 100)

dict_1 = {}
for i in X.columns:
    modl = xgb.XGBRegressor().fit(X_train[i], y_train)
    dict_1[i] = modl.get_booster().get_score().values()
    
keys = list(dict_1.keys())
values = list(dict_1.values())
data_kv = pd.DataFrame(data=values, index = keys, columns = ['вес']).sort_values(by = 'вес', ascending = False)
data_kv.head()

,вес
Total Income from Entrepreneurial Acitivites,3096.0
Bread and Cereals Expenditure,2914.0
Total Rice Expenditure,2902.0
Total Food Expenditure,2859.0
Total Fish and marine products Expenditure,2835.0


# Корреляция 

In [206]:
corr_matrix = X[data_kv[data_kv['вес']>0].index].corr(method = 'pearson')
corr_matrix.style.background_gradient(cmap = 'viridis').set_precision(4)

,Total Income from Entrepreneurial Acitivites,Bread and Cereals Expenditure,Total Rice Expenditure,Total Food Expenditure,Total Fish and marine products Expenditure,Restaurant and hotels Expenditure,Vegetables Expenditure,Medical Care Expenditure,Crop Farming and Gardening expenses,"Clothing, Footwear and Other Wear Expenditure",Tobacco Expenditure,Alcoholic Beverages Expenditure,Meat Expenditure,Education Expenditure,Housing and water Expenditure,Fruit Expenditure,Transportation Expenditure,Special Occasions Expenditure,Miscellaneous Goods and Services Expenditure,Communication Expenditure,House Floor Area,Imputed House Rental Value,House Age,Household Head Age,Total Number of Family members,Number of bedrooms,Number of Cellular phone,Total number of family members employed,Members with age 5 - 17 years old,Number of Motorcycle/Tricycle,Number of CD/VCD/DVD,Number of Television,Number of Component/Stereo set,Number of Personal Computer,Members with age less than 5 year old,Number of Refrigerator/Freezer,Number of Airconditioner,Number of Landline/wireless telephones,"Number of Car, Jeep, Van",Number of Washing Machine,Number of Motorized Banca,Number of Stove with Oven/Gas Range,Agricultural Household indicator,Electricity
Total Income from Entrepreneurial Acitivites,1.0000,0.1376,0.0959,0.2099,0.2000,0.1401,0.1812,0.1130,0.3161,0.2671,0.0348,0.0883,0.2046,0.1992,0.2095,0.2030,0.2444,0.1645,0.2041,0.2630,0.1378,0.2148,0.0053,0.0204,0.0685,0.1491,0.1550,-0.1139,0.0136,0.1450,0.1176,0.1809,0.1140,0.1883,-0.0068,0.1945,0.2357,0.1261,0.3262,0.1430,0.0444,0.1377,0.0286,0.0348
Bread and Cereals Expenditure,0.1376,1.0000,0.8763,0.6203,0.3914,0.1725,0.3492,0.0556,0.0852,0.2244,0.1488,0.1310,0.3468,0.1594,0.2186,0.2289,0.2196,0.1468,0.2403,0.2050,0.1324,0.0957,-0.0185,-0.0031,0.6128,0.1873,0.3608,0.3009,0.3823,0.1676,0.1604,0.1795,0.1058,0.1731,0.2108,0.1369,0.1009,0.0744,0.1064,0.1365,0.0298,0.0956,-0.0253,0.0833
Total Rice Expenditure,0.0959,0.8763,1.0000,0.4779,0.2777,0.0913,0.2333,0.0242,0.0862,0.1193,0.1358,0.0925,0.2150,0.0976,0.1314,0.1177,0.1227,0.0943,0.1384,0.0931,0.0899,0.0241,-0.0332,-0.0012,0.5464,0.1398,0.2824,0.2584,0.3504,0.1371,0.1014,0.1181,0.0551,0.0816,0.1824,0.0681,0.0276,0.0184,0.0390,0.0686,0.0303,0.0396,-0.0373,0.0829
Total Food Expenditure,0.2099,0.6203,0.4779,1.0000,0.5963,0.5994,0.5509,0.1634,0.0269,0.4710,0.1774,0.2102,0.7116,0.3236,0.6532,0.4651,0.4896,0.2661,0.5244,0.5366,0.3255,0.3294,0.0952,-0.0388,0.4853,0.3140,0.6050,0.3517,0.2214,0.2608,0.3713,0.4633,0.2681,0.4403,0.1391,0.4290,0.3201,0.2388,0.2906,0.4633,0.0181,0.2834,0.0218,0.2576
Total Fish and marine products Expenditure,0.2000,0.3914,0.2777,0.5963,1.0000,0.2073,0.4856,0.1157,0.0521,0.3244,0.1261,0.1675,0.4654,0.2131,0.3254,0.3565,0.3072,0.2047,0.3327,0.3242,0.1716,0.1932,0.0278,0.0418,0.3140,0.2110,0.3376,0.1728,0.1226,0.1872,0.2027,0.2464,0.1718,0.2662,0.0644,0.2478,0.2251,0.1555,0.2158,0.2149,0.0682,0.1857,-0.0036,0.0579
Restaurant and hotels Expenditure,0.1401,0.1725,0.0913,0.5994,0.2073,1.0000,0.1975,0.1157,-0.0477,0.4001,0.0848,0.1220,0.4350,0.3290,0.4833,0.2821,0.4945,0.1754,0.4624,0.5391,0.2092,0.3284,0.1204,0.0027,0.1478,0.1790,0.4191,0.2557,0.0051,0.1067,0.2611,0.3273,0.1974,0.3892,-0.0298,0.2997,0.3179,0.2502,0.2933,0.3657,-0.0218,0.2014,0.0876,0.1648
Vegetables Expenditure,0.1812,0.3492,0.2333,0.5509,0.4856,0.1975,1.0000,0.1214,0.0888,0.3296,0.0797,0.1467,0.4743,0.2075,0.3411,0.4382,0.3033,0.1867,0.3243,0.3313,0.2087,0.2080,0.0340,0.0636,0.2891,0.2151,0.3297,0.1749,0.1142,0.1767,0.1921,0.2540,0.1617,0.2805,0.0605,0.2520,0.2064,0.1535,0.2172,0.2500,0.0362,0.1943,0.0263,0.0717
Medical Care Expenditure,0.1130,0.0556,0.0242,0.1634,0.1157,0.1157,0.1214,1.0000,0.0101,0.1728,0.0068,0.0171,0.1636,0.0888,0.2293,0.1777,0.2004,0.1859,0.2317,0.2147,0.1532,0.1782,0.0821,0.1033,0.0008,0.1646,0.1305,0.0245,-0.0717,0.0609,0.1150,0.1938,0.1123,0.1774,-0.0051,0.1864,0.1994,0.1184,0.1780,0.1684,-0.0047,0.1521,-0.0329,0.0755
Crop Farming and Gardening ex

In [207]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

threshold = 0.6
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('%d features могут быть удалены' % (len(to_drop)))

11 features могут быть удалены


In [208]:
X = X[data_kv[data_kv['вес']>0].index]

In [209]:
X = X.drop(columns = to_drop)

# Перебор

In [210]:
# GridSearchCV если позволяет мощность
#%%time
param_grid  = { 
                'n_estimators' : [500,200],
                'max_depth': range(2,5,1),
                 #'max_features' : ('log2', 'sqrt'),
                 #'class_weight':[{1: w} for w in [1,1.5]]
              }

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 100)


GridRF = GridSearchCV(xgb.XGBRegressor(random_state=15), param_grid, scoring = 'r2') #neg_mean_squared_error
#GridRF = GridSearchCV(RandomForestRegressor(random_state=15), param_grid, scoring = 'r2') #neg_mean_squared_error

GridRF.fit(X_train, y_train)

print(
     "\nBest parameters \n" + str(GridRF.best_params_))


Best parameters 
{'max_depth': 3, 'n_estimators': 200}


In [211]:
feature_importances = np.zeros(df.shape[1])

model = xgb.XGBRegressor(random_state=42,**GridRF.best_params_)
#model = RandomForestRegressor(random_state=42,**GridRF.best_params_)


In [212]:
model.fit(X_train, y_train)

feature_importances = model.feature_importances_

In [213]:
feature_importances = model.feature_importances_ 
feature_importances = pd.DataFrame({'feature': list(X.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)

feature_importances.head()

,feature,importance
14,Miscellaneous Goods and Services Expenditure,0.225988
22,Number of Television,0.121565
12,Transportation Expenditure,0.088059
28,Number of Washing Machine,0.080040
18,Number of bedrooms,0.046273


In [214]:
lst = []
for i in list(np.unique(feature_importances[feature_importances['importance'] > 0.03]['feature'])):
    lst.append(i)

In [215]:
X_train, X_test, y_train, y_test = train_test_split(X[lst], y, test_size = 0.25, random_state = 100)

In [216]:
%%time
param_grid  = { 
                'n_estimators' : [100,200],
                'max_depth': range(2,4,1),
                #'max_features' : ('log2', 'sqrt'),
                #'class_weight':[{1: w} for w in [1,1.5]]
              }

GridRF = GridSearchCV(xgb.XGBRegressor(random_state=15), param_grid)

GridRF.fit(X_train, y_train)

print(
     "\nBest parameters \n" + str(GridRF.best_params_))


Best parameters 
{'max_depth': 3, 'n_estimators': 100}
Wall time: 8.6 s


In [217]:
clf_2 = xgb.XGBRegressor(random_state=42,**GridRF.best_params_)
#clf_2 = RandomForestRegressor(random_state=42,**GridRF.best_params_)
clf_2.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [218]:
clf_score = cross_val_score(clf_2, X_train, y_train, cv=10)
print(clf_score)
clf_score.mean()

[0.73429426 0.73551619 0.75051578 0.74575759 0.73757012 0.74228594
 0.74429761 0.73666804 0.74351203 0.74663726]


0.7417054812987918

In [219]:
clf_score = cross_val_score(clf_2, X_test, y_test, cv=10)
print(clf_score)
clf_score.mean()

[0.72738512 0.70177815 0.74233448 0.73101522 0.74591607 0.72599368
 0.7218033  0.7602212  0.76041725 0.72471092]


0.7341575391540106

In [220]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = clf_2.predict(X_train)
r2 = r2_score(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
print("r2 TRAIN: %.2f" % r2)
print("MSE TRAIN: %.2f" % mse)

r2 TRAIN: 0.77
MSE TRAIN: 0.10


In [221]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = clf_2.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print("r2 TRAIN: %.2f" % r2)
print("MSE TRAIN: %.2f" % mse)

r2 TRAIN: 0.74
MSE TRAIN: 0.11


# 

In [222]:
df_model_ = dataset.copy()
df_model_.head()

,Total Household Income,Total Food Expenditure,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,Vegetables Expenditure,Restaurant and hotels Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,480332,117848,0,42140,38300,24676,16806,3325,13460,3000,...,1,1,0,0,0,2,1,0,0,1
1,198235,67766,0,17329,13008,17434,11073,2035,7833,2360,...,0,1,0,0,0,3,1,0,0,2
2,82785,61609,1,34182,32001,7783,2590,1730,3795,4545,...,0,0,0,0,0,0,0,0,0,0
3,107589,78189,0,34030,28659,10914,10812,690,7887,6280,...,0,0,0,0,0,1,0,0,0,0
4,189322,94625,0,34820,30167,18391,11309,1395,11260,6400,...,1,0,0,0,0,3,0,0,0,1


In [223]:
df_model_.shape

(41544, 45)

In [224]:
X.shape

(41544, 33)

In [225]:
df_model_['Model'] = clf_2.predict(X_train + X_test)
df_model_['INCOME_y'] = y

In [226]:
pd.DataFrame(df_model_['INCOME_y']).describe()

,INCOME_y
count,41544.000000
mean,11.996651
std,0.653172
min,9.331230
25%,11.560715
50%,12.008105
75%,12.449834
max,13.447486


In [227]:
pd.DataFrame(df_model_['Model'] ).describe()

,Model
count,41544.000000
mean,9.884812
std,0.003879
min,9.888691
25%,9.888691
50%,9.888691
75%,9.888691
max,9.888691


In [228]:
df_model_['delta2'] = [abs(x-y)/x*100 if x!=0 else 0 for x,y in zip(df_model_['INCOME_y'], df_model_['Model'])]
df_model_['delta_2_'] = [abs(x-y)/x*100 if (x-y)>0 else 0. for x,y in zip(df_model_['INCOME_y'], df_model_['Model'])]

In [244]:
def q_1(x):
    return x.quantile(0.1)

def q_2(x):
    return x.quantile(0.9)

def q_mean(x):
    return x.quantile(0.5)

def func_income(tmp, income, coun):
    tmp_1 = tmp.copy()
    tmp_1['delta_1'] = tmp_1['delta_2_']
    tmp_1['delta_2'] = tmp_1['delta_2_']
    tmp_1['delta_3'] = tmp_1['delta_2_']
    bins = np.arange(0,15,0.5)
    bins = np.arange(0,tmp_1[income].max(),tmp_1[income].sort_values(ascending = True).quantile(0.005))
    bins = np.arange(0,tmp_1[income].max(),0.1)
    tmp_1[income+'_bucket'] = [bins[np.digitize(x, bins) - 1] for x in tmp_1[income]]
    
    tmp_2 = tmp_1.groupby(by = [income+'_bucket']).agg({'delta_2_':'mean', coun: 'count',
                                                         'delta_1': q_1, 'delta_2': q_2, 'delta_3': q_mean}).reset_index()
    x = tmp_2[income+'_bucket']
    y = tmp_2['delta_3']
    z = tmp_2['delta_1']
    q = tmp_2['delta_2']
    y_1 = tmp_2['delta_2_']
    w = tmp_2[coun]
    
    fig = go.Figure()
    fig =make_subplots(specs = [[{'secondary_y':True}]])
    fig.add_trace(go.Scatter(x = x, y = z, name = '10%_left', fill=None), secondary_y = True)
    fig.add_trace(go.Scatter(x = x, y = q, name = '10%_right', fill='tonexty'), secondary_y = True)
    fig.add_trace(go.Scatter(x = x, y = y, name = 'median'), secondary_y = True)
    fig.add_trace(go.Scatter(x = x, y = y_1, name = 'mean'), secondary_y = True)
    fig.add_trace(go.Bar(x = x, y = w, name = 'count'))
    
    fig.update_layout(yaxis2 = dict(range = [0,100]))
    fig.show()
    
    return tmp_2

In [245]:
tmp = func_income(df_model_, 'INCOME_y', 'delta2')